In [4]:
import pandas as pd
from tqdm import tqdm
import json

# 计算所在行政区邻近性

In [14]:
district_adj_matrix_file = './raw_data/beijing_district_adjacency.csv'
poi_district_file = './raw_data/district.data'
poi_district_adj_matrix_file = './output_data/beijing_poi_district_adj_matrix.data'
shortest_driving_dis_adj_matrix_file = './output_data/shortest_driving_dis_adj_matrix.data'
shortest_driving_time_adj_matrix_file = './output_data/shortest_driving_time_adj_matrix.data'
least_texi_fee_adj_matrix_file = './output_data/least_texi_fee_adj_matrix.data'

In [5]:
district_adj_matrix = pd.read_csv(district_adj_matrix_file, index_col=0)
district_adj_matrix

,东城区,西城区,朝阳区,丰台区,石景山区,海淀区,顺义区,通州区,大兴区,房山区,门头沟区,昌平区,平谷区,密云区,怀柔区,延庆区
东城区,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
西城区,1,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0
朝阳区,1,1,2,1,0,1,1,1,1,0,0,1,0,0,0,0
丰台区,1,1,1,2,1,1,0,0,1,1,1,0,0,0,0,0
石景山区,0,0,0,1,2,1,0,0,0,0,1,0,0,0,0,0
海淀区,0,1,1,1,1,2,0,0,0,0,1,1,0,0,0,0
顺义区,0,0,1,0,0,0,2,1,0,0,0,1,1,1,1,0
通州区,0,0,1,0,0,0,1,2,1,0,0,0,0,0,0,1
大兴区,0,0,1,1,0,0,0,1,2,1,0,0,0,0,0,0
房山区,0,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0


In [6]:
poi_district = {}
with open(poi_district_file, mode='r', encoding='utf8') as f:
    for line in f:
        line = line.strip('\n')
        if line == '':
            continue
        line = line.split('\t')
        poi_district[line[0]] = line[1]
print(len(poi_district))

137


In [7]:
poi_list = list(poi_district.keys())
poi_district_adj_matrix = pd.DataFrame(index=poi_list, columns=poi_list)
poi_district_adj_matrix[poi_list] = 0
poi_district_adj_matrix

,万方亭公园,万科石京龙滑雪场,世界公园,世界花卉大观园,世界葡萄博览中心景区,中国人民抗日战争纪念馆,中国园林博物馆,中国地质博物馆,中央广播电视塔,中山公园,...,静之湖度假区,颐和园,首都博物馆,香山公园,鹿世界主题园景区,黄花水长城旅游区,黑龙潭旅游区,龙庆峡,龙潭湖公园,龙脉温泉度假村
万方亭公园,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
万科石京龙滑雪场,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
世界公园,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
世界花卉大观园,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
世界葡萄博览中心景区,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
黄花水长城旅游区,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
黑龙潭旅游区,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
龙庆峡,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
龙潭湖公园,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
for i in tqdm(range(len(poi_list))):
    for j in range(len(poi_list)):
        if i == j:
            poi_district_adj_matrix[poi_list[i]][poi_list[j]] = 99
        elif poi_district[poi_list[i]] == poi_district[poi_list[j]]:
            poi_district_adj_matrix[poi_list[i]][poi_list[j]] = 2
        elif district_adj_matrix[poi_district[poi_list[i]]][poi_district[poi_list[j]]] == 1:
            poi_district_adj_matrix[poi_list[i]][poi_list[j]] = 1
poi_district_adj_matrix

100%|██████████| 137/137 [00:03<00:00, 35.99it/s]


,万方亭公园,万科石京龙滑雪场,世界公园,世界花卉大观园,世界葡萄博览中心景区,中国人民抗日战争纪念馆,中国园林博物馆,中国地质博物馆,中央广播电视塔,中山公园,...,静之湖度假区,颐和园,首都博物馆,香山公园,鹿世界主题园景区,黄花水长城旅游区,黑龙潭旅游区,龙庆峡,龙潭湖公园,龙脉温泉度假村
万方亭公园,99,0,2,2,0,2,2,1,1,1,...,0,1,1,1,0,0,1,0,1,0
万科石京龙滑雪场,0,99,0,0,2,0,0,0,0,0,...,1,0,0,0,1,0,0,2,0,1
世界公园,2,0,99,2,0,2,2,1,1,1,...,0,1,1,1,0,0,1,0,1,0
世界花卉大观园,2,0,2,99,0,2,2,1,1,1,...,0,1,1,1,0,0,1,0,1,0
世界葡萄博览中心景区,0,2,0,0,99,0,0,0,0,0,...,1,0,0,0,1,0,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
黄花水长城旅游区,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,99,0,0,0,0
黑龙潭旅游区,1,0,1,1,0,1,1,1,2,0,...,1,2,1,2,0,0,99,0,0,1
龙庆峡,0,2,0,0,2,0,0,0,0,0,...,1,0,0,0,1,0,0,99,0,1
龙潭湖公园,1,0,1,1,0,1,1,1,0,2,...,0,0,1,0,0,0,0,0,99,0


In [30]:
poi_district_adj_matrix.to_csv(poi_district_adj_matrix_file)

# 三种距离矩阵

In [9]:
poi_list = list(poi_district.keys())
# 最短驾驶距离
shortest_driving_dis_adj_matrix = pd.DataFrame(index=poi_list, columns=poi_list)
shortest_driving_dis_adj_matrix[poi_list] = 0
shortest_driving_dis_adj_matrix

# 最短驾驶时间
shortest_driving_time_adj_matrix = pd.DataFrame(index=poi_list, columns=poi_list)
shortest_driving_time_adj_matrix[poi_list] = 0
shortest_driving_time_adj_matrix

# 最少打车费用
least_texi_fee_adj_matrix = pd.DataFrame(index=poi_list, columns=poi_list)
least_texi_fee_adj_matrix[poi_list] = 0
least_texi_fee_adj_matrix

for poi in poi_list:
    shortest_driving_dis_adj_matrix[poi][poi] = 1
    shortest_driving_time_adj_matrix[poi][poi] = 1
    least_texi_fee_adj_matrix[poi][poi] = 1

In [10]:
shortest_driving_dis_adj_matrix

,万方亭公园,万科石京龙滑雪场,世界公园,世界花卉大观园,世界葡萄博览中心景区,中国人民抗日战争纪念馆,中国园林博物馆,中国地质博物馆,中央广播电视塔,中山公园,...,静之湖度假区,颐和园,首都博物馆,香山公园,鹿世界主题园景区,黄花水长城旅游区,黑龙潭旅游区,龙庆峡,龙潭湖公园,龙脉温泉度假村
万方亭公园,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
万科石京龙滑雪场,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
世界公园,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
世界花卉大观园,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
世界葡萄博览中心景区,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
黄花水长城旅游区,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
黑龙潭旅游区,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
龙庆峡,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
龙潭湖公园,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
with open('./output_data/geographic_info.data.full', mode='r', encoding='utf8') as f:
    for line in tqdm(f):
        line = line.strip('\n')
        if line == '':
            continue
        line = json.loads(line)
        ori_name = line['ori_name']
        des_name = line['des_name']
        time = line['shortest_driving_time']
        distance = line['shortest_driving_path']
        fee = line['texi_fee']
        shortest_driving_dis_adj_matrix[ori_name][des_name] = 1/(distance**2)
        shortest_driving_dis_adj_matrix[des_name][ori_name] = 1/(distance**2)
        shortest_driving_time_adj_matrix[ori_name][des_name] = 1/(time**2)
        shortest_driving_time_adj_matrix[des_name][ori_name] = 1/(time**2)
        least_texi_fee_adj_matrix[ori_name][des_name] = 1/(fee**2)
        least_texi_fee_adj_matrix[des_name][ori_name] = 1/(fee**2)
        

0it [00:00, ?it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_25324\3832073926.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  least_texi_fee_adj_matrix[ori_name][des_name] = 1/(fee**2)
9316it [00:01, 6989.07it/s]


In [18]:
shortest_driving_dis_adj_matrix.loc[:,(shortest_driving_dis_adj_matrix==0).any()].columns

Index([], dtype='object')

In [19]:
shortest_driving_time_adj_matrix.loc[:,(shortest_driving_time_adj_matrix==0).any()].columns

Index([], dtype='object')

In [20]:
least_texi_fee_adj_matrix.loc[:,(least_texi_fee_adj_matrix==0).any()].columns

Index([], dtype='object')

三个矩阵都不存在缺失值

In [17]:
shortest_driving_dis_adj_matrix.to_csv(shortest_driving_dis_adj_matrix_file)
shortest_driving_time_adj_matrix.to_csv(shortest_driving_time_adj_matrix_file)
least_texi_fee_adj_matrix.to_csv(least_texi_fee_adj_matrix_file)